# SVM
In this notebook, I provide a sample implementation of the SVM model for comparative analysis. Spectra are first obtained from files in the "dataset" folder.

In [ ]:
# Dependencies
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load and preprocess data
def load_and_preprocess(train_file, val_file):
    """Load and preprocess training and validation data."""
    train_df = pd.read_csv(train_file, header=None)
    val_df = pd.read_csv(val_file, header=None)
    
    # Extract samples and labels
    train_samples = train_df.iloc[1:-1, 1:].values.astype(float)
    train_labels = train_df.iloc[-1, 1:].values.astype(float)
    val_samples = val_df.iloc[1:-1, 1:].values.astype(float)
    val_labels = val_df.iloc[-1, 1:].values.astype(float)
    
    # Standardize features
    scaler = StandardScaler()
    train_samples = scaler.fit_transform(train_samples.T).T
    val_samples = scaler.transform(val_samples.T).T
    
    return train_samples, train_labels, val_samples, val_labels

In [ ]:
# Metrics storage
all_metrics = []

# Train and evaluate the model 30 times across all seeds
for seed_index in range(30):
    print(f"Seed {seed_index+1}/30")

    # Load and preprocess data
    train_file = f"../No_MSC/Seed_{seed_index}/Combined_Train.csv"
    val_file = f"../No_MSC/Seed_{seed_index}/Combined_Validation.csv"
    train_samples, train_labels, val_samples, val_labels = load_and_preprocess(train_file, val_file)
    
    # Prepare data for SVM (flattened samples)
    X_train = train_samples.T
    y_train = train_labels
    X_val = val_samples.T
    y_val = val_labels

    # Define SVM with RBF kernel
    svm = SVC(kernel='rbf', C=100.0, gamma='scale', probability=True)

    # Train the SVM model
    svm.fit(X_train, y_train)

    # Predict on the validation set
    y_pred = svm.predict(X_val)

    # Calculate evaluation metrics
    val_accuracy = accuracy_score(y_val, y_pred)
    val_precision = precision_score(y_val, y_pred)
    val_recall = recall_score(y_val, y_pred)
    val_f1 = f1_score(y_val, y_pred)

    # Store metrics
    all_metrics.append((val_accuracy, val_precision, val_recall, val_f1))

In [ ]:
# Convert the metrics to a DataFrame for easier plotting
metrics_df = pd.DataFrame(all_metrics, columns=['val_accuracy', 'val_precision', 'val_recall', 'val_f1'])

# Plot the box plots for each metric
plt.figure(figsize=(12, 8))

# Box plot for validation accuracy
plt.subplot(2, 3, 1)
sns.boxplot(y=metrics_df['val_accuracy'])
plt.title('Validation Accuracy')
plt.xlabel('Accuracy')
mean_val = metrics_df['val_accuracy'].mean()
plt.scatter(0, mean_val, color='red', s=100, zorder=10)

# Box plot for validation precision
plt.subplot(2, 3, 2)
sns.boxplot(y=metrics_df['val_precision'])
plt.title('Validation Precision')
plt.xlabel('Precision')
mean_val = metrics_df['val_precision'].mean()
plt.scatter(0, mean_val, color='red', s=100, zorder=10)

# Box plot for validation recall
plt.subplot(2, 3, 3)
sns.boxplot(y=metrics_df['val_recall'])
plt.title('Validation Recall')
plt.xlabel('Recall')
mean_val = metrics_df['val_recall'].mean()
plt.scatter(0, mean_val, color='red', s=100, zorder=10)

# Box plot for validation F1 score
plt.subplot(2, 3, 4)
sns.boxplot(y=metrics_df['val_f1'])
plt.title('Validation F1 Score')
plt.xlabel('F1 Score')
mean_val = metrics_df['val_f1'].mean()
plt.scatter(0, mean_val, color='red', s=100, zorder=10)

# Adjust layout and show plots
plt.tight_layout()
plt.show()

In [ ]:
# Save the metrics for later comparison
metrics_df.to_csv("cenMetrics/SVM_metrics.csv", index=False)

print("Finished Cross-Validation")